In [7]:
import pandas as pd
import datetime as dt

# Step 1: Load the Excel file
file_path = '/content/online_retail_II.xlsx'
xlsx = pd.ExcelFile(file_path)

In [8]:
# Show available sheets
print("Sheet names:", xlsx.sheet_names)

Sheet names: ['Year 2009-2010', 'Year 2010-2011']


In [9]:
# Load sheet_1
df_1 = pd.read_excel(xlsx, sheet_name='Year 2009-2010')

In [10]:
# Load sheet_2
df_2 = pd.read_excel(xlsx, sheet_name='Year 2010-2011')

In [11]:
df_1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [12]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [13]:
df_2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [14]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [15]:
# Combine the datasets
df = pd.concat([df_1, df_2], ignore_index=True)

In [16]:
# Drop rows with missing Customer ID or Description
df.dropna(subset=['Customer ID', 'Description'], inplace=True)

In [17]:
# Convert Customer ID to string
df['Customer ID'] = df['Customer ID'].astype(str)

In [18]:
# Remove rows with negative or zero Quantity or Price
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]

In [19]:
# Remove duplicates
df.drop_duplicates(inplace=True)

In [20]:
# Create TotalPrice column
df['Total Price'] = df['Quantity'] * df['Price']

In [21]:
# Remove canceled invoices (usually start with 'C')
df = df[~df['Invoice'].astype(str).str.startswith('C')]

In [22]:
# Set reference date for Recency (day after last transaction)
ref_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

In [28]:
# Save as Excel to current working directory
excel_path = 'cleaned_online_retail.xlsx'
df.to_excel(excel_path, index=False)

# Download the file
from google.colab import files
files.download(excel_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
# Load cleaned data
df = pd.read_excel('cleaned_online_retail.xlsx')

In [30]:
# RFM Table
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (ref_date - x.max()).days,   # Recency
    'Invoice': 'nunique',                                # Frequency
    'Total Price': 'sum'                                  # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

In [31]:
# Score each metric
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4,3,2,1])  # Lower = better
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1,2,3,4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1,2,3,4])

# Combine to create RFM segment code
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].sum(axis=1).astype(int)

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Normalize RFM values
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# K-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

In [34]:
# Save RFM table to CSV in the current Colab directory
rfm.to_excel('online_retail_rfm.xlsx', index=False)

# Download it to your local machine
from google.colab import files
files.download('online_retail_rfm.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>